# Manual Feature Engineering

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

In [ ]:
bureau=pd.read_csv('../input/bureau.csv')
bureau.head()

In [ ]:
previous_loan_counts=bureau.groupby('SK_ID_CURR',as_index=False)['SK_ID_BUREAU'].count().rename(columns={'SK_ID_BUREAU':'previous_loan_counts'})

previous_loan_counts.head()


In [ ]:
train=pd.read_csv('../input/application_train.csv')
train=train.merge(previous_loan_counts,on='SK_ID_CURR',how='left')

train['previous_loan_counts']=train['previous_loan_counts'].fillna(0)
train.head()

# Assessing Usefulness of New Variabel with r value

- Pearson Correlation Coefficient(r-value) between this variable and the target.

## Kernel Density Estimate Plots

In [ ]:
def kde_target(var_name,df):
    
    corr=df['TARGET'].corr(df[var_name])
    
    avg_repaid=df.ix[df['TARGET']==0,var_name].median()
    avg_not_repaid=df.ix[df['TARGET']==1,var_name].median()
    
    plt.figure(figsize=(12,6))
    
    sns.kdeplot(df.ix[df['TARGET']==0,var_name],label='target==0')
    sns.kdeplot(df.ix[df['TARGET']==1,var_name],label='target==1')
    
    plt.title('Density of %s by Target Value'%var_name)
    plt.xlabel('%s'%var_name)
    plt.ylabel('Density')
    plt.legend()
    
    print('The correlation between %s and the Target is %0.4f'%(var_name,corr))
    print('Median value of %s for repaid= %0.4f'%(var_name,avg_repaid))
    print('Median value of %s for not repaid= %0.4f'%(var_name,avg_not_repaid))
    


In [ ]:
kde_target('EXT_SOURCE_3',train)

In [ ]:
kde_target('previous_loan_counts',train)

# Aggregating numerical 

In [ ]:
bureau_agg=bureau.drop(columns=['SK_ID_BUREAU']).groupby('SK_ID_CURR',as_index=False).agg(['count','mean','max','min','sum']).reset_index()
bureau_agg.head()

In [ ]:
columns=['SK_ID_CURR']

for var in bureau_agg.columns.levels[0]:
    if var !='SK_ID_CURR':
        for stat in bureau_agg.columns.levels[1][:-1]:
            columns.append('bureau_%s_%s'%(var,stat))

In [ ]:
bureau_agg.columns=columns
bureau_agg.head()

In [ ]:
train=train.merge(bureau_agg,on='SK_ID_CURR',how='left')
train.head()

## Correlation of Aggregated Values with Target

In [ ]:
new_corrs=[]

for col in columns:
    corr=train['TARGET'].corr(train[col])
    
    new_corrs.append((col,corr))

In [ ]:
new_corrs=sorted(new_corrs,key=lambda x: abs(x[1]),reverse=True)
new_corrs[:15]

In [ ]:
kde_target('bureau_DAYS_CREDIT_mean',train)

# Function for Numerica Aggregations

In [ ]:
def agg_numeric(df,group_var,df_name):
    for col in df:
        if col!=group_var and 'SK_ID' in col:
            df=df.drop(columns=col)
    
    group_ids=df[group_var]
    numeric_df=df.select_dtypes('number')
    numeric_df[group_var]=group_ids
    
    agg=numeric_df.groupby(group_var).agg(['count','mean','max','min','sum']).reset_index()
    
    columns=[group_var]
    
    for var in agg.columns.levels[0]:
        if var!=group_var:
            for stat in agg.columns.levels[1][:-1]:
                columns.append('%s_%s_%s'%(df_name,var,stat))
    agg.columns=columns
    return agg

In [ ]:
bureau_agg_new=agg_numeric(bureau.drop(columns=['SK_ID_BUREAU']),group_var='SK_ID_CURR',df_name='bureau')
bureau_agg_new.head()

In [ ]:
bureau_agg.head()

In [ ]:
def target_corrs(df):
    
    corrs=[]
    
    for col in df.columns:
        print(col)
        if col != 'TARGET':
            corr=df['TARGET'].corr(df[col])
            
            corrs.append(corr)
            
    corrs=sorted(corrs,key=lambda x: abs(x[1]),reverse=True)
    
    return corrs

# Categorical Variables

In [ ]:
categorical=pd.get_dummies(bureau.select_dtypes('object'))
categorical['SK_ID_CURR']=bureau['SK_ID_CURR']
categorical.head()

In [ ]:
categorical_grouped=categorical.groupby('SK_ID_CURR').agg(['sum','mean'])
categorical_grouped.head()

In [ ]:
categorical_grouped.columns.levels[0]

In [ ]:
categorical_grouped.columns.levels[1]

In [ ]:
group_var='SK_ID_CURR'

columns=[]

for var in categorical_grouped.columns.levels[0]:
    if var!=group_var:
        for stat in ['count','count_sum']:
            columns.append('%s_%s'%(var,stat))
            
categorical_grouped.columns=columns

categorical_grouped.head()

In [ ]:
train=train.merge(categorical_grouped,on='SK_ID_CURR',how='left')
train.head()

In [ ]:
train.shape

In [ ]:
train.iloc[:10,123:]

## Function to Handle Categorical Variables

In [ ]:
def count_categorical(df,group_var,df_name):
    categorical=pd.get_dummies(df.select_dtypes('object'))
    categorical[group_var]=df[group_var]
    
    categorical=categorical.groupby(group_var).agg(['sum','mean'])
    
    columns=[]
    
    for var in categorical.columns.levels[0]:
        if var!=group_var:
            for stat in ['count','count_norm']:
                columns.append('%s_%s_%s'%(df_name,var,stat))
    
    categorical.columns=columns
    
    return categorical

In [ ]:
bureau_counts=count_categorical(bureau,'SK_ID_CURR','bureau')
bureau_counts.head()

## Applying Operations to another dataframe

In [ ]:
bureau_balance=pd.read_csv('../input/bureau_balance.csv')
bureau_balance.head()

In [ ]:
bureau_balance_counts=count_categorical(bureau_balance,group_var='SK_ID_BUREAU',df_name='bureau_balance')
bureau_balance_counts.head()

In [ ]:
bureau_balance_agg=agg_numeric(bureau_balance,group_var='SK_ID_BUREAU',df_name='bureau_balance')
bureau_balance_agg.head()

In [ ]:
bureau_by_loan=bureau_balance_agg.merge(bureau_balance_counts,right_index=True, left_on='SK_ID_BUREAU',how='outer')

bureau_by_loan=bureau_by_loan.merge(bureau[['SK_ID_BUREAU','SK_ID_CURR']],on='SK_ID_BUREAU',how='left')

bureau_by_loan.head()

In [ ]:
bureau_balance_by_client=agg_numeric(bureau_by_loan.drop(columns=['SK_ID_BUREAU']),group_var='SK_ID_CURR',df_name='client')

bureau_balance_by_client.head()

In bureau_balance dataframe:
1. Calculate numeric stats grouping by each loan
2. Made value counts of each categorical variable grouping by loan
3. Merge the stats and value counts on the loans
4. Calculated numeric stats for the resulting dataframe grouping by the client id

Final resulting dataframe has one row for each client with statistics calculated for all of their loans with monthly balance information

## Put functions together

delect the former variables and recalculate these variables from the groudup

In [ ]:
import gc
gc.enable()
del train,bureau,bureau_balance,bureau_agg,bureau_agg_new,bureau_balance_agg,bureau_balance_counts, bureau_by_loan,bureau_balance_by_client,bureau_counts
gc.collect()

In [ ]:
train=pd.read_csv('../input/application_train.csv')
bureau=pd.read_csv('../input/bureau.csv')
bureau_balance=pd.read_csv('../input/bureau_balance.csv')

## Counts of Bureau Dataframe

In [ ]:
bureau_counts=count_categorical(bureau,group_var='SK_ID_CURR',df_name='bureau')
bureau_counts.head()

## Aggregated Stats of Bureau Dataframe

In [ ]:
bureau_agg=agg_numeric(bureau.drop(columns='SK_ID_BUREAU'),group_var='SK_ID_CURR',df_name='bureau')
bureau_agg.head()

## Value counts of Bureau Balance dataframe by loan

In [ ]:
bureau_balance_counts=count_categorical(bureau_balance,group_var='SK_ID_BUREAU',df_name='bureau_balance')
bureau_balance_counts.head()

## Aggregated stats of Bureau Balance dataframe by loan

In [ ]:
bureau_balance_agg=agg_numeric(bureau_balance, group_var='SK_ID_BUREAU', df_name='bureau_balance')
bureau_balance_agg.head()

## Aggregated Stats of Bureau Balance by Client

In [ ]:
bureau_by_loan=bureau_balance_agg.merge(bureau_balance_counts, right_index=True,left_on='SK_ID_BUREAU',how='outer')
bureau_by_loan=bureau[['SK_ID_BUREAU','SK_ID_CURR']].merge(bureau_by_loan,on='SK_ID_BUREAU',how='left')

bureau_balance_by_client=agg_numeric(bureau_by_loan.drop(columns=['SK_ID_BUREAU']),group_var='SK_ID_CURR',df_name='client')

## Insert Computed Features into Training Data

In [ ]:
original_features=list(train.columns)
print('Original Number of Features: ', len(original_features))

In [ ]:
#Merge with value counts of bureau
train=train.merge(bureau_counts,on='SK_ID_CURR',how='left')

#Merge with the stats of bureau
train=train.merge(bureau_agg,on='SK_ID_CURR',how='left')

#Merge with the monthly information grouped by client
train=train.merge(bureau_balance_by_client, on='SK_ID_CURR',how='left')

In [ ]:
new_features=list(train.columns)
print('Number of features using previous loans from other institutions data: ', len(new_features))

# Feature Engineering Outcomes

After all that work, now we want to take a look at the variables we have created. We can look at the percentage of missing values, the correlations of variables with the target, and also the correlation of variables with the other variabels. The correlations between variables can show if we have colinear variables, that is , variables that are highly correlated with one another. Often, we want to remove one in a pair of colinear variables because having both variables would be redundant. We can also use the percentage of missing values to remove features with a substantial majority of values that are not present.

**Feature selection** will be an important focus going forward, because reducing the number of features can help the model learn during training and also generalize better to the testing data. The "curse of dimensionality'' is the name given to the issues caused by having too many features (too high of a dimension). As the number of variables increases, the number of datapoints needed to learn the relationship between these variables and the target value increase exponentially.

Feature selection is the process of removing variables to help our model to learn and generalize better to the testing set. The objective is to remove useless/redundant variables while preserving those that useful. These are a number of tools we can use for this process, but in this notebook we will stick to removing columns with a high percentage of missing values and variables that have a high correlation with one another. Later we can look at using the features importances returned from models such as the Gradient Boosting Machine or Random Forest to perform feature selection.

## Missing Values
An important consideration is the missing values in the dataframe. Columns with too many missing values might have to be dropped.

In [ ]:
# Function to caluculate missing values by column
def missing_values_table(df):
    
    #Total missing values
    mis_val=df.isnull().sum()
    
    #Percentage of missing values
    mis_val_percent=100*df.isnull().sum()/len(df)
    
    #Make a table with the results
    mis_val_table=pd.concat([mis_val,mis_val_percent],axis=1)
    
    #Rename the columns
    mis_val_table_ren_columns=mis_val_table.rename(columns={0:'Missing Values',1:'% of Total Values'})
    
    #Sort th table by percentage of missing descending 
    mis_val_table_ren_columns=mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1]!=0].sort_values('% of Total Values', ascending=False).round(1)
    
    # print some summary information
    print('Your selected dataframe has '+str(df.shape[1])+ " columns.\n" "There are "+str(mis_val_table_ren_columns.shape[0])+' columns that have missing values.')
    
    return mis_val_table_ren_columns

In [ ]:
missing_train=missing_values_table(train)
missing_train.head(10)

We see that there are number of columns with a high percentage of missing values. There is no well-established threshold for removing missing values, and the best course of action depends on the problem. Here, to reduce the number of features, we will remove any columns in either the training or the testing data that have greater than 90% missing values.

In [ ]:
missing_train_vars=list(missing_train.index[missing_train['% of Total Values']>90.0])
len(missing_train_vars)

Before we remove the missing values, we will find the missing value percentages in the testing data. We will then remove any columns with greater than 90% missing values in either the training or testing data. Let's now read in the testing data, perform the same operations, and look at the missing values in the testing data. We already have caluculated all the counts and aggregation statistics, so we only need to merge the testing data with the appropriate data.

## Calculating information for Testing Data

In [ ]:
#Reading the test dataframe
test=pd.read_csv('../input/application_test.csv')

#Merge with the stats of bureau
test=test.merge(bureau_counts,on='SK_ID_CURR',how='left')

#Merge with the value counts of bureau balance
test=test.merge(bureau_agg,on='SK_ID_CURR',how='left')

#Merge with the balue counts of bureau balance
test=test.merge(bureau_balance_by_client, on='SK_ID_CURR',how='left')

In [ ]:
print('Shape of Testing Data: ',test.shape)

We need  to align the testing and training dataframes, which means matching up the columns so they have the exact  same columns, This shouldn't be an issue here, but when we one-hot encode variables, we need to align the dataframes to make sure they have the same columns.

In [ ]:
missing_test=missing_values_table(test)
missing_test.head(10)

In [ ]:
missing_test_vars=list(missing_test.index[missing_test['% of Total Values']>90])
len(missing_test_vars)

In [ ]:
missing_columns=list(set(missing_test_vars+missing_train_vars))
len(missing_columns)
print('There are %d columns with more than 90%% missing in either the training or testing data.'%len(missing_columns))

In [ ]:
train=train.drop(columns=missing_columns)
test=test.drop(columns=missing_columns)

We ended up removing no  columns in this round because there are no columns with more than 90% missing values. We might have to apply another feature selection method to reduce the dimensionality.

At this point we will save both the training and testing data. I encourage anyone to try different percentages for dropping the missing columns and compare the outcomes.

In [ ]:
train.to_csv('train_bureau_raw.csv',index=False)
test.to_csv('test_bureau_raw.csv',index=False)

## Correlations
First let's look at the correlations of the variables with the target. We can see in any of the varaibles we created have a greater correlation than those already present in the training data(from application)

In [ ]:
# Calculate all correlations in dataframe
corrs=train.corr()

In [ ]:
corrs=corrs.sort_values('TARGET',ascending=False)

#Ten most positive correlations
pd.DataFrame(corrs['TARGET'].head(10))

In [ ]:
pd.DataFrame(corrs['TARGET'].dropna().tail(10))

The highest correlated variable with the target (other than the TARGET which of course has a correlation of 1), is a variable we created. However, just because the variable is correlated does not mean that ti will be useful, and we have to remember that **if we  generate hundreds of new variables, some are going to be correlated with the target simpy because of random noise.**

Viewing the correlations skeptically, it does appear that several of the newly created variables may be useful. **To assess the 'usefulness' of variables, we will look at the feature importances return by the model.** For curiousity's sake (and because we already wrote the function) we can make a kde plot of two of the newly created variables.

In [ ]:
kde_target(var_name='client_bureau_balance_MONTHS_BALANCE_min_mean',df=train)

In [ ]:
kde_target(var_name='bureau_CREDIT_ACTIVE_Active_count_norm',df=train)

Well this last distribution is all over the place. This variable represents the number of previous loans with a CREDIT_ACTIVE value of Active divided by the total number of previous loans for a client. The correlation here is so weak that I do not think we should draw any conclusions!

### Colinear Variables
We can calculate not only the correlations of the variables with the target, but also the correlation of each variable with every other variable. This will allow us to see if there are highly colinear variables that should perhaps be removed from the data.

Let's look for any variables that have a greater than 0.8 correlation with other variables.

In [ ]:
# Set the threshold
threshold = 0.8

# Empty dictionary to hold correlated variables
above_threshold_vars={}

#For each column, record the variables that are above the threshold
for col in corrs:
    above_threshold_vars[col]=list(corrs.index[corrs[col]>threshold])

For each of these pairs of highly corrlated variables, we only want to remove one of the variables. The following code creates a set of variables to remove by only adding one of each pair.

In [ ]:

# Track columns to remove  and columns aleardy examined
cols_to_remove=[]
cols_seen=[]
cols_to_remove_pair=[]
# Iterate through columns and correlated columns
for key,value in above_threshold_vars.items():
    # Keep track of the columns already examined
    cols_seen.append(key)
    for x in value:
        if x == key:
            next
        else:
            #only want to remove one in a pair
            if x not in cols_seen:
                cols_to_remove.append(x)
                cols_to_remove_pair.append(key)
                
cols_to_remove=list(set(cols_to_remove))
print('Number of columns to remove: ',len(cols_to_remove))

We can remove these columns from both the training and the testing datasets. We will have to compare performance after removing these variables with performance keeping these variables (the raw csv files we saved earlier).

In [ ]:
train_corrs_removed=train.drop(columns=cols_to_remove)
test_corrs_removed=test.drop(columns=cols_to_remove)

print('Training Corrs Removed Shape: ',train_corrs_removed.shape)
print('Testing Corrs Removed Shape: ',test_corrs_removed.shape)

In [ ]:
train_corrs_removed.to_csv('train_bureau_corrs_removed.csv',index=False)
test_corrs_removed.to_csv('test_bureau_corrs_removed.csv',index=False)


# Modeling
To actually test the performance of these new datasets, we will try using them for machine learning! Here we will use a function I developed in another notebook to compare the features (the raw version with the highly correlated variables removed). We can run this kind of like an experiment, and the contril will be the performance of just the application data in this function when submitted to the competition. I've already recorded that performance, so we can list out our control and our two test conditions:

**For all datasets, use the model shown below (with the exact hyperparameters)**

- control: only the data in the application files.
- test one: the data in applcation files with all of the data recorded from the bureau and bureau_balance files
- test two: the data in the application files with all of the data recordedd from the bureau and bureau_balance files with highly correlated varaibles removed.

In [ ]:
import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

import gc

import matplotlib.pyplot as plt

In [ ]:
def model(features, test_features, encoding='ohe', n_folds=5):
    """
    Train and test a light gradient boosting model using cross validation.
    

    
    """
    
    train_ids=features['SK_ID_CURR']
    test_ids=test_features['SK_ID_CURR']
    
    labels=features['TARGET']
    
    features=features.drop(columns=['SK_ID_CURR','TARGET'])
    test_features=test_features.drop(columns=['SK_ID_CURR'])
    
    
    if encoding=='ohe':
        features=pd.get_dummies(features)
        test_features=pd.get_dummies(test_features)
        
        features, test_features=features.align(test_features,join='inner',axis=1)
        
        cat_indices='auto'
        
    elif encoding=='le':
        
        label_encoder=labelEncoder()
        
        cat_indices=[]
        
        for i,col in enumerate(features):
            if features[col].dtype=='object':
                features[col]=label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col]=label_encoder.fit_transform(np.array(test_features[col].astype(str)).reshape((-1,)))
                
                cat_indices.append(i)
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
    
    print('Training Data shape: ',features.shape)
    print('Testing Data shape: ',test_features.shape)
    
    feature_names=list(features.columns)
    
    features=np.array(features)
    test_features=np.array(test_features)
    
    #Create the kfold object
    k_fold=KFold(n_splits=n_folds,shuffle=False,random_state=50)
    
    #Empty array for feature importances
    feature_importance_values=np.zeros(len(feature_names))
    
    #Empty array for test predictions
    test_predictions=np.zeros(test_features.shape[0])
    
    #Empty array for out of fold validation predictions
    out_of_fold=np.zeros(features.shape[0])
    
    #Lists for recording validation and training scores
    valid_scores=[]
    train_scores=[]
    
    #Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        #Training data for the fold
        train_features,train_labels=features[train_indices],labels[train_indices]
        
        #Validation data for the fold
        valid_features,valid_labels=features[valid_indices],labels[valid_indices]
        
        #Create the model
        model=lgb.LGBMClassifier(n_estimators=10000,objective='binary', class_weight='balanced', learning_rate=0.05, reg_alpha=0.1, reg_lambda=0.1, subsample=0.8,n_jobs=-1,random_state=50)
        
        #Train the model
        model.fit(train_features, train_labels, eval_metric='auc',eval_set=[(valid_features, valid_labels),(train_features,train_labels)], eval_names=['valid','train'],categorical_feature=cat_indices, early_stopping_rounds=100, verbose=200)
        
        # Record the best iteration
        best_iteration=model.best_iteration_
        
        #Record the feature importances
        feature_importance_values+=model.feature_importances_/k_fold.n_splits
        
        #Make predictions
        test_predictions+=model.predict_proba(test_features,num_iteration=best_iteration)[:,1]/k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices]=model.predict_proba(valid_features,num_iteration=best_iteration)[:,1]
        
        #Record the best score
        valid_score=model.best_score_['valid']['auc']
        train_score=model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model,train_features, valid_features
        gc.collect()
        
    submission=pd.DataFrame()